In [114]:
import pickle
import fastapy
import numpy as np
import re


alphaToID: dict[str, int] = {
    "A": 0,
    "B": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "J": 9,
    "K": 10,
    "L": 11,
    "M": 12,
    "N": 13,
    "O": 14,
    "P": 15,
    "Q": 16,
    "R": 17,
    "S": 18,
    "T": 19,
    "U": 20,
    "V": 21,
    "W": 22,
    # "X": 23,
    "Y": 23,
    "Z": 24,
}

specialChar = ("X", "*", "-")

vec_len = 25 * 25


def encode(s: str) -> int:
    assert len(s) == 2
    first = s[0]
    second = s[1]

    return alphaToID[first] * 25 + alphaToID[second]


def get_vect(s: str) -> np.ndarray:
    vect = np.zeros(vec_len, dtype=np.int32)
    for i in range(len(s) - 1):
        if s[i] in specialChar or s[i + 1] in specialChar:
            continue

        bigram = s[i : i + 2]
        id = encode(bigram)
        vect[id] += 1
    return vect
    


In [84]:
def build_database(filename: str):
    database: list[tuple[str, np.ndarray]] = []

    for i, record in enumerate(fastapy.parse(filename)):
        if i % 1000 == 0:
            print(i)

        current_db_entry = get_vect(record.seq)
        database.append((record.id, current_db_entry))
        
    # Save the resulting `database` to a pickle file
    with open(f"{filename}_database.pkl", "wb") as f:
        pickle.dump(database, f)


In [85]:
def build_id_names(filename: str):
    id_name_dict = {}

    for i, record in enumerate(fastapy.parse(filename)):
        if i % 10000 == 0:
            print(i)

        match = re.match( r'>\w+\|\w+\|\w+ (.*)OS=(.*) O.*SV=\d+$', record.description)    
        id_name_dict[record.id] = "Protein: " + match.group(1) + " || Organism: " + match.group(2)

    # Save the resulting `id_name_dict` to a pickle file
    with open(f"{filename}_names.pkl", "wb") as f:
        pickle.dump(id_name_dict, f)

In [86]:
# cosine vector similarity
def cosine_similarity(v1: np.ndarray, v2: np.ndarray) -> float:
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [87]:
# max heap to find top 100 most similar sequences
import heapq

def find_top_100_similar_sequences(database: list[tuple[str, np.ndarray]], target_sequence: str) -> list[tuple[float, str]]:
    target_vector = get_vect(target_sequence)
    similarities = []
    for entry in database:
        similarity = cosine_similarity(entry[1], target_vector)
        heapq.heappush(similarities, (-similarity, entry[0]))
        if len(similarities) > 100:
            heapq.heappop(similarities)
    return [(-similarity, entry) for similarity, entry in similarities]


In [88]:
def print_names(proteins: list[tuple[float, str]], names):    
    for i, protein in enumerate(proteins):
        print(f"{i+1}: {names[protein[1]]} ({protein[0]}) [{protein[1]}]")

# Testing

In [89]:
test_filename = "uniprot-choline_esterase_reviewed-yes.fasta"

In [90]:
build_database(test_filename)
build_id_names(test_filename)

0
0


In [91]:
with open(f"{test_filename}_database.pkl", "rb") as f:
    test_database = pickle.load(f)

# print the first 3 entries
print(test_database[:3])


[('sp|P06276|CHLE_HUMAN', array([2, 0, 0, 0, 2, 5, 2, 0, 1, 0, 2, 5, 0, 2, 0, 4, 2, 2, 1, 3, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 2, 1, 2, 0, 3, 0, 0, 1, 1,
       2, 0, 1, 3, 0, 0, 0, 0, 1, 2, 2, 0, 5, 0, 0, 2, 1, 4, 2, 1, 4, 0,
       0, 1, 2, 2, 0, 0, 1, 2, 4, 2, 0, 0, 4, 0, 0, 1, 0, 0, 1, 1, 3, 6,
       3, 1, 0, 3, 7, 0, 2, 0, 3, 2, 0, 2, 2, 0, 2, 2, 1, 0, 3, 0, 1, 3,
       1, 5, 4, 0, 1, 0, 4, 4, 1, 6, 0, 1, 1, 1, 3, 4, 0, 3, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 1, 0, 1, 0,
       1, 0, 2, 0, 2, 2, 1, 1, 1, 0, 5, 0, 1, 6, 1, 0, 0, 2, 0, 1, 0, 1,
       0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 3, 3, 2, 0, 0, 1, 0, 4, 3, 0, 4,
       0, 2, 0, 1, 3, 1, 0, 3, 1, 3, 0, 4, 0, 1, 0, 4, 5, 5, 2, 2, 0, 1,
       5, 0, 4, 0, 3, 2, 

In [92]:
with open(f"{test_filename}_names.pkl", "rb") as f:
    test_names = pickle.load(f)

# print the test_names
print(test_names)

{'sp|P06276|CHLE_HUMAN': 'Protein: Cholinesterase  || Organism: Homo sapiens', 'sp|Q03311|CHLE_MOUSE': 'Protein: Cholinesterase  || Organism: Mus musculus', 'sp|P81908|CHLE_HORSE': 'Protein: Cholinesterase  || Organism: Equus caballus', 'sp|Q04958|NTE1_YEAST': 'Protein: Lysophospholipase NTE1  || Organism: Saccharomyces cerevisiae (strain ATCC 204508 / S288c)', 'sp|P21927|CHLE_RABIT': 'Protein: Cholinesterase  || Organism: Oryctolagus cuniculus', 'sp|P32749|CHLE_BOVIN': 'Protein: Cholinesterase  || Organism: Bos taurus', 'sp|O62760|CHLE_FELCA': 'Protein: Cholinesterase  || Organism: Felis catus', 'sp|O62761|CHLE_PANTT': 'Protein: Cholinesterase  || Organism: Panthera tigris tigris', 'sp|P32752|CHLE_PIG': 'Protein: Cholinesterase (Fragment)  || Organism: Sus scrofa', 'sp|P32750|CHLE_CANLF': 'Protein: Cholinesterase (Fragment)  || Organism: Canis lupus familiaris', 'sp|P32751|CHLE_MACMU': 'Protein: Cholinesterase (Fragment)  || Organism: Macaca mulatta', 'sp|P32753|CHLE_SHEEP': 'Protein:

In [93]:
# sp|P21927|CHLE_RABIT
t = "MVTRSSHTEDVIITTKNGRIRGINLPVFGGTVTAFLGIPYAQPPLGRLRFKKPQSLTKWS" + \
"DIWNATKYANSCCQNIDQSFPGFHGSEMWNPNTDLSEDCLYLNVWIPTPKPKNATVMIWI" + \
"YGGGFQTGTSSLQVYDGKFLTRVERVIVVSMNYRVGALGFLALPGNPEAPGNMGLFDQQL" + \
"ALQWVQKNIAAFGGNPKSVTLFGESAGAASVSLHLLSPRSHPLFTRAILQSGSSNAPWEV" + \
"MSLHEARNRTLTLAKFVGCSTENETEIIKCLRNKDAQEILLNEVFVVPFDSLLSVNFGPT" + \
"VDGDFLTDMPDTLLQLGQLKKTQILVGVNKDEGTAFLVYGAPGFSKDNTSIITRKEFQEG" + \
"LKIFFPGVSEFGKESILFHYTDWVDEQRPENYREALDDVVGDYNFICPALEFTKKFSEWG" + \
"NNAFFYYFEHRSSKLPWPEWMGVMHGYEIEFVFGLPLERRVNYTKAEEILSRSIMKRWAN" + \
"FAKYGNPNGTQNNSTRWPVFKSTEQKYLTLNTESPRIYTKLRAQQCRFWTLFFPKVLEMT" + \
"GNIDEAEQEWKAGFHRWNNYMMAWKNHFNDYTSKKERCAGF"


In [94]:
find_top_100_similar_sequences(test_database, t)

[(np.float64(0.9999999999999999), 'sp|P21927|CHLE_RABIT'),
 (np.float64(0.9393618312248179), 'sp|P06276|CHLE_HUMAN'),
 (np.float64(0.9393322193908735), 'sp|P81908|CHLE_HORSE'),
 (np.float64(0.9066362235267058), 'sp|O62761|CHLE_PANTT'),
 (np.float64(0.8864442568763394), 'sp|Q03311|CHLE_MOUSE'),
 (np.float64(0.9361723273087207), 'sp|P32749|CHLE_BOVIN'),
 (np.float64(0.9088570181304345), 'sp|O62760|CHLE_FELCA'),
 (np.float64(0.7282902199862923), 'sp|Q9NDG8|ACE4_CAEBR'),
 (np.float64(0.773922004527753), 'sp|Q9DDE3|ACES_DANRE'),
 (np.float64(0.7540361096000756), 'sp|P07692|ACES_TORMA'),
 (np.float64(0.739166902309156), 'sp|A4QVZ8|NTE1_MAGO7'),
 (np.float64(0.7313560503916586), 'sp|Q5BAE9|NTE1_EMENI'),
 (np.float64(0.7386672384529938), 'sp|Q869C3|ACES_ANOGA'),
 (np.float64(0.7436215270075182), 'sp|Q7S8J1|NTE1_NEUCR'),
 (np.float64(0.7319364674887895), 'sp|Q9USJ4|NTE1_SCHPO'),
 (np.float64(0.7209490820805731), 'sp|Q5A368|NTE1_CANAL'),
 (np.float64(0.7202191933412805), 'sp|Q6FKJ1|NTE1_CANGA'),

In [95]:
print_names(find_top_100_similar_sequences(test_database, t), test_names)

1: Protein: Cholinesterase  || Organism: Oryctolagus cuniculus (0.9999999999999999) [sp|P21927|CHLE_RABIT]
2: Protein: Cholinesterase  || Organism: Homo sapiens (0.9393618312248179) [sp|P06276|CHLE_HUMAN]
3: Protein: Cholinesterase  || Organism: Equus caballus (0.9393322193908735) [sp|P81908|CHLE_HORSE]
4: Protein: Cholinesterase  || Organism: Panthera tigris tigris (0.9066362235267058) [sp|O62761|CHLE_PANTT]
5: Protein: Cholinesterase  || Organism: Mus musculus (0.8864442568763394) [sp|Q03311|CHLE_MOUSE]
6: Protein: Cholinesterase  || Organism: Bos taurus (0.9361723273087207) [sp|P32749|CHLE_BOVIN]
7: Protein: Cholinesterase  || Organism: Felis catus (0.9088570181304345) [sp|O62760|CHLE_FELCA]
8: Protein: Acetylcholinesterase 4  || Organism: Caenorhabditis briggsae (0.7282902199862923) [sp|Q9NDG8|ACE4_CAEBR]
9: Protein: Acetylcholinesterase  || Organism: Danio rerio (0.773922004527753) [sp|Q9DDE3|ACES_DANRE]
10: Protein: Acetylcholinesterase  || Organism: Torpedo marmorata (0.75403610

# Building a database

In [96]:
filename = "uniprot_sprot.fasta"
build_database(filename)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [97]:
build_id_names(filename)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000


# Main tests

In [98]:
import numpy as np
import pickle

In [99]:
# load a file database from pickle
with open(f"{filename}_database.pkl", "rb") as f:
    database = pickle.load(f)

# print the first 10 entries
print(database[:10])


[('sp|Q6GZX4|001R_FRG3G', array([1, 0, 0, 0, 1, 2, 0, 1, 0, 0, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 1, 1, 0, 2, 0, 1, 3, 0,
       0, 0, 0, 0, 2, 1, 0, 0, 2, 0, 1, 0, 1, 0, 1, 1, 0, 0, 2, 0, 0, 0,
       3, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 3, 2, 0, 1, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 3, 0, 1, 0, 1, 2, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 3, 3, 2, 3, 1, 3, 0, 1, 1, 1, 0,
       0, 0, 2, 1, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 2, 1, 0, 1, 0, 2, 0, 3,
       4, 0, 0, 0, 0, 0, 

In [100]:
# load names from pickle
with open(f"{filename}_names.pkl", "rb") as f:
    names = pickle.load(f)

In [101]:
print_names(find_top_100_similar_sequences(database, t), names)

1: Protein: [D-Met2]-deltorphin  || Organism: Phasmahyla jandaia (0.05059029115727206) [sp|P86634|DELT_PHAJA]
2: Protein: Contryphan-Fic  || Organism: Conus figulinus (0.04956815970966096) [sp|P0DP14|COWC_CONFI]
3: Protein: Electrin-3  || Organism: Litoria rubella (0.04956815970966096) [sp|P82099|EI03_LITRU]
4: Protein: Cryptide Pep-22  || Organism: Tityus obscurus (0.04956815970966096) [sp|P0DRG7|CRY22_TITOB]
5: Protein: Corazonin  || Organism: Striatophasma naukluftense (0.04702448523165576) [sp|B0M3C2|CORZ_STRNA]
6: Protein: Unknown protein 1 (Fragment)  || Organism: Capsicum chinense (0.04956815970966096) [sp|P86083|UP01_CAPCH]
7: Protein: FMRFamide-3  || Organism: Sarcophaga bullata (0.04956815970966096) [sp|P85476|FAR3_SARBU]
8: Protein: Corazonin  || Organism: Austrophasma gansbaaiense (0.04702448523165576) [sp|B3A0F4|CORZ_AUSGA]
9: Protein: 34 kDa cell wall protein (Fragment)  || Organism: Arabidopsis thaliana (0.04956815970966096) [sp|P80831|CWP07_ARATH]
10: Protein: Corazonin

In [ ]:
# your Protein
t = ""
print_names(find_top_100_similar_sequences(database, t), names)